# **Exploratory Data Analysis (EDA)**

In [1]:
#Import libraries
import pandas as pd

In [2]:
#Open the dataset and check the first lines of code
df_business = pd.read_parquet("df_business.parquet")

In [13]:
ny_count = (df_business['state'] == 'NY').sum()
print(ny_count)

0


In [14]:
unique_states = df_business['state'].unique()
print(unique_states)

['CA' 'PA' 'FL' 'TX']


In [3]:
df_users = pd.read_csv('df_user.csv')
df_users.head()

,Unnamed: 0,user_id,name,review_count,useful,id_user_yelp
0,0,qVc8ODYU5SZjKXVBgXdI7w,Walker,585,7217,0
1,1,j14WgRoU_-2ZE1aw1dXrJg,Daniel,4333,43091,1
2,2,2WnXYQFK0hXEoTxPtV2zvg,Steph,665,2086,2
3,3,SZDeASXq7o05mMNLshsdIA,Gwen,224,512,3
4,4,hA5lMy-EnncsH4JoR-hFGQ,Karen,79,29,4


In [4]:
df_reviews = pd.read_parquet('df_reviews.parquet')
df_reviews.head()

,review_id,user_id,business_id,stars,text,date,id_review_yelp
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3.0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11,0
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5.0,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18,1
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3.0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30,2
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5.0,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03,3
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4.0,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15,4


In [5]:
# Merge df_reviews and df_business
combined_df = pd.merge(df_reviews, df_business, on='business_id', how='left')

# Merge the result with df_users
combined_df = pd.merge(combined_df, df_users, on='user_id', how='left')

In [6]:
# Convert the 'date' column to datetime
combined_df['date'] = pd.to_datetime(combined_df['date'])

# Filter to keep only rows with date from 2016 onwards
combined_df = combined_df[combined_df['date'].dt.year >= 2016]
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4596987 entries, 0 to 6990279
Data columns (total 22 columns):
 #   Column          Dtype         
---  ------          -----         
 0   review_id       object        
 1   user_id         object        
 2   business_id     object        
 3   stars_x         float64       
 4   text            object        
 5   date            datetime64[ns]
 6   id_review_yelp  int64         
 7   name_x          object        
 8   address         object        
 9   city            object        
 10  state           object        
 11  postal_code     object        
 12  latitude        float64       
 13  longitude       float64       
 14  stars_y         float64       
 15  review_count_x  float64       
 16  categories      object        
 17  Unnamed: 0      float64       
 18  name_y          object        
 19  review_count_y  float64       
 20  useful          float64       
 21  id_user_yelp    float64       
dtypes: datetime64[ns](1), f

In [7]:
combined_df.head(20)

,review_id,user_id,business_id,stars_x,text,date,id_review_yelp,name_x,address,city,...,latitude,longitude,stars_y,review_count_x,categories,Unnamed: 0,name_y,review_count_y,useful,id_user_yelp
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3.0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11,0,Turning Point of North Wales,1460 Bethlehem Pike,North Wales,...,40.210196,-75.223639,3.0,169.0,None,124234.0,Melanie,33.0,32.0,124234.0
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4.0,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15,4,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,150111.0,Sophia,126.0,74.0,150111.0
8,ZKvDG2sBvHVdF5oBNUOpAQ,wSTuiTk-sKNdcFyprzZAjg,B5XSoSG3SfvQGtKEGQ1tSQ,3.0,This easter instead of going to Lopez Lake we ...,2016-03-30 22:46:33,8,Los Padres National Forest,San Marcos Pass Rd,Santa Barbara,...,34.597239,-119.510772,4.5,13.0,None,154296.0,Reyli,24.0,31.0,154296.0
9,pUycOfUwM8vqX7KjRRhUEA,59MxRhNVhU9MYndMkz0wtw,gebiRewfieSdtt17PTW6Zg,3.0,Had a party of 6 here for hibachi. Our waitres...,2016-07-25 07:31:06,9,Hibachi Steak House & Sushi Bar,502 State St,Santa Barbara,...,34.416984,-119.695556,3.5,488.0,None,114367.0,Joe,12.0,3.0,114367.0
18,u2vzZaOqJ2feRshaaF1doQ,NDZvyYHTUWWu-kqgQzzDGQ,CLEWowfkj-wKYJlQDqT1aw,5.0,I go to blow bar to get my brows done by natal...,2016-03-07 00:02:18,18,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,80127.0,Johana,27.0,69.0,80127.0
24,lUUhg8ltDsUZ9h0xnwY4Dg,RreNy--tOmXMl1en0wiBOg,cPepkJeRMtHapc_b2Oe_dw,4.0,I was really between 3 and 4 stars for this on...,2018-07-17 03:30:07,24,Naked Tchopstix Express,"2902 W 86th St, Ste 70",Indianapolis,...,39.912505,-86.211285,3.5,33.0,None,81195.0,Patricia,240.0,264.0,81195.0
27,z0osLHDvXvzfm57D4DmD2Q,xVKE_HJ2pwUtTdLbL3pnCg,S2Ho8yLxhKAa26pBAm6rxA,3.0,"Service was crappy, and food was mediocre. I ...",2016-11-22 00:22:53,27,Creole House Restaurant & Oyster Bar,509 Canal St,New Orleans,...,29.952213,-90.067116,4.0,1594.0,None,123114.0,Megan,31.0,32.0,123114.0
30,-P5E9BYUaK7s3PwBF5oAyg,Jha0USGDMefGFRLik_xFQg,bMratNjTG5ZFEA6hVyr-xQ,5.0,First time there and it was excellent!!! It fe...,2017-02-19 13:32:05,30,Portobello Cafe,1423 Chester Pike,Eddystone,...,39.865032,-75.344051,4.0,137.0,"Restaurants, Seafood, Cafes, Italian",113321.0,Lauren,116.0,75.0,113321.0
31,YbMyvlDA2W3Py5lTz8VK-A,4hBhtCSgoxkrFgHa4YAD-w,bbEXAEFr4RYHLlZ-HFssTA,5.0,"Great burgers,fries and salad! Burgers have a...",2017-01-02 03:17:34,31,The Original Habit Burger Grill,5735 Hollister Ave,Goleta,...,34.435570,-119.824706,4.0,329.0,"Fast Food, Burgers, Restaurants",42869.0,Heidi,922.0,1687.0,42869.0
32,40thYphUgIfvJq17QCfTwA,QzCEzH3R7Z6erOGLr3t55Q,0pMj5xUAecW9o1P35B0AMw,5.0,Great staff always helps and always nice. Alwa...,2017-05-26 13:10:24,32,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,60114.0,Kylhalil,14.0,5.0,60114.0


In [8]:
# Obtaining data only of "restaurant" category
combined_df = combined_df[combined_df['categories'].str.lower().str.contains('restaurant', na=False)].reset_index(drop=True)
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 534325 entries, 0 to 534324
Data columns (total 22 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   review_id       534325 non-null  object        
 1   user_id         534325 non-null  object        
 2   business_id     534325 non-null  object        
 3   stars_x         534325 non-null  float64       
 4   text            534325 non-null  object        
 5   date            534325 non-null  datetime64[ns]
 6   id_review_yelp  534325 non-null  int64         
 7   name_x          534325 non-null  object        
 8   address         534325 non-null  object        
 9   city            534325 non-null  object        
 10  state           534325 non-null  object        
 11  postal_code     534325 non-null  object        
 12  latitude        534325 non-null  float64       
 13  longitude       534325 non-null  float64       
 14  stars_y         534325 non-null  flo

In [10]:
# Delete the columns 'Unnamed: 0'
combined_df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [11]:
#Rename columns
combined_df.rename(columns={'name_x': 'business_name', 'name_y': 'user_name'}, inplace=True)

combined_df.rename(columns={'review_count_x': 'review_count_business', 'review_count_y': 'review_count_user'}, inplace=True)

combined_df.rename(columns={'stars_x': 'stars_business', 'stars_y': 'stars_user'}, inplace=True)

In [12]:
# Save the new data
ruta_parquet = 'df_yelp.parquet'

# Guardar el DataFrame user_items en el archivo Parquet especificado
combined_df.to_parquet(ruta_parquet, index=False)

print(f'The file was succesfully saved in: {ruta_parquet}')

The file was succesfully saved in: df_yelp.parquet
